<a href="https://colab.research.google.com/github/kiyong21c/pytorch_tutorial/blob/main/20220706_Dataset_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data Loader

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

# gradient computation etc. not efficient for whole data set
# -> divide dataset into small batches

'''
# training loop
for epoch in range(num_epochs):
    # loop over all batches
    for i in range(total_batches):
        batch_x, batch_y = ...
'''

# epoch = one forward and backward pass of ALL training samples
# batch_size = number of training samples used in one forward/backward pass
# number of iterations = number of passes, each pass (forward+backward) using [batch_size] number of sampes
# e.g : 100 samples, batch_size=20 -> 100/20=5 iterations for 1 epoch

# --> DataLoader can do the batch computation for us

# Implement a custom Dataset:
# inherit Dataset
# implement __init__ , __getitem__ , and __len__

class WineDataset(Dataset):
    # Dataset 클래스를 상속받으면서, 1)__getitem__, 2)__len__ 재정의(오버라이딩)
    def __init__(self):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('/content/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0] # 데이터 수

        # here the first column is the class label, the rest are the features
        self.x_data = torch.from_numpy(xy[:, 1:]) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(xy[:, [0]]) # size [n_samples, 1], [0]을 사용한 이유는 Series가 아닌 DataFrame을 만들기 위함

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        # print('__getitem__ 메서드 호출됨')
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        # print('__len__ 메서드 호출됨')
        return self.n_samples


# create dataset
dataset = WineDataset()

# get first sample and unpack
first_data = dataset[0] # class 매직메소드인 __getitem__이 호출됨
features, labels = first_data
print('-'*10, '첫번째 데이터셋', '-'*10)
print('features :',features, 'labels :',labels)

# Load whole dataset with DataLoader
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses
# !!! IF YOU GET AN ERROR DURING LOADING, SET num_workers TO 0 !!!
train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=2)

# convert to an iterator and look at one random sample
dataiter = iter(train_loader) # train_loader를 출력하기위해
data = dataiter.next()
features, labels = data # train_loader를 features, labels로 반환할 수 있음
print('-'*10, 'train_loader(shuffled) sample(random batch)', '-'*10)
print(features, labels) # batch_size에 따라 4개의 데이터셋을 가짐

# Dummy Training loop
num_epochs = 2
total_samples = len(dataset)    # class 매직메소드인 __len__이 호출됨
n_iterations = math.ceil(total_samples/4)   # total_samples(178)/batch_size(4)를 올림
print('전체 데이터 개수 :', total_samples, ', 전체 데이터를 학습하기 위한 batch의 반복횟수 :', n_iterations)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader): # i는 iteration이 되겠고, inputs는 피처, labels는 타겟
        



        # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
        # Run your training process
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')

# # some famous datasets are available in torchvision.datasets
# # e.g. MNIST, Fashion-MNIST, CIFAR10, COCO

# train_dataset = torchvision.datasets.MNIST(root='./data', 
#                                            train=True, 
#                                            transform=torchvision.transforms.ToTensor(),  
#                                            download=True)

# train_loader = DataLoader(dataset=train_dataset, 
#                                            batch_size=3, 
#                                            shuffle=True)

# # look at one random sample
# dataiter = iter(train_loader)
# data = dataiter.next()
# inputs, targets = data
# print(inputs.shape, targets.shape)

---------- 첫번째 데이터셋 ----------
features : tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) labels : tensor([1.])
---------- train_loader(shuffled) sample(random batch) ----------
tensor([[1.3170e+01, 2.5900e+00, 2.3700e+00, 2.0000e+01, 1.2000e+02, 1.6500e+00,
         6.8000e-01, 5.3000e-01, 1.4600e+00, 9.3000e+00, 6.0000e-01, 1.6200e+00,
         8.4000e+02],
        [1.1840e+01, 8.9000e-01, 2.5800e+00, 1.8000e+01, 9.4000e+01, 2.2000e+00,
         2.2100e+00, 2.2000e-01, 2.3500e+00, 3.0500e+00, 7.9000e-01, 3.0800e+00,
         5.2000e+02],
        [1.1870e+01, 4.3100e+00, 2.3900e+00, 2.1000e+01, 8.2000e+01, 2.8600e+00,
         3.0300e+00, 2.1000e-01, 2.9100e+00, 2.8000e+00, 7.5000e-01, 3.6400e+00,
         3.8000e+02],
        [1.2340e+01, 2.4500e+00, 2.4600e+00, 2.1000e+01, 9.8000e+01, 2.5600e+00,
         2.1100e+00, 3.4000e-01, 1.3100e+00, 2.8000e+00,